# 0. Импорт и загрузка данных

## 0.1 Импорт

In [1]:
from scipy.sparse import csr_matrix, csc_matrix, coo_matrix, find
from matplotlib.cm import ScalarMappable
from matplotlib.colors import Normalize
from townsnet import Region, Provision
from collections import defaultdict
import matplotlib.pyplot as plt
from tqdm.auto import tqdm
from pprint import pprint
from pathlib import Path
import geopandas as gpd
import pandas as pd
import osmnx as ox
import numpy as np
import itertools
import folium
import os

from utils.base_model import BaseModel


from shapely.geometry import LineString, Point
import random

from utils.ueqi import calculate_ueqi, count_change_capacity, UEQI_GROUPS
from utils.planter import *

np.random.seed(0)
random.seed(0)

## 0.2 Функции

In [2]:
# Обновление населения
def update_population(region, towns_df, path='data/population.csv'):
    population = pd.read_csv(path)

    indexes = towns_df[towns_df.town_name.isin(population.region_city.to_list())].index.tolist() # Выделим все города, в которых можем обновить население
    for i in indexes: # Пройдемся по все городам
        region[i].population = population[population['region_city'] == region[i].name].population.iloc[0] # Обновим

In [3]:
# Метод-заглушка. Добавляет случайный сервис из подходящей группы в случайном месте города
def update_service(region, name, new_capacity, params):
    service_key = random.choice(list(params.keys()))
    type_of_service = random.choice(UEQI_GROUPS[service_key])[9:]
    services = region.get_services_gdf()
    towns = region.get_towns_gdf()
    service = services[(services.town.str.contains(name)) & (services.service_type == type_of_service)][['geometry', 'capacity']]
    town_id = towns[towns.town_name.str.contains(name)].index.item()
    service['town_id'] = town_id

    # Получаем координаты центроида города
    town_geometry = towns[towns.town_name.str.contains(name)].geometry.item()
    center_x = town_geometry.x
    center_y = town_geometry.y

    # Генерируем случайное смещение от -300 до +300 метров
    offset_x = random.uniform(-300, 300)
    offset_y = random.uniform(-300, 300)

    # Создаём точку с учетом смещения
    new_service_point = Point(center_x + offset_x, center_y + offset_y)

    new_services = gpd.GeoDataFrame(
        {
            'town_id': town_id,
            'geometry': [new_service_point],
            'capacity': [new_capacity],
        },
        crs=region.crs  # Должно совпадать с CRS региона
    )

    service = pd.concat([service, new_services])        # объединяем данные
    region.update_services(type_of_service, service)    # обновляем

    return type_of_service

In [4]:
# Получение значений UEQI для города
def get_ueqi(wff, name):
    city = wff.cities[wff.cities['region_city'].str.contains(name)]
    return city[
                ['region_city',
                "ueqi_residential",
                "ueqi_street_networks",
                "ueqi_green_spaces",
                "ueqi_public_and_business_infrastructure",
                "ueqi_social_and_leisure_infrastructure",
                "ueqi_citywide_space"]
            ].round(3)

In [5]:
# Загрузка матриц миграции
def load_matrices(infra_keys, total_nodes, matrix_dir="data/provision", use_updated=False, average=True):
    if isinstance(infra_keys, str):
        keys_flat = [infra_keys]
    else:
        keys_flat = list(itertools.chain.from_iterable(
            [k] if isinstance(k, str) else k for k in infra_keys
        ))

    # Проверка ключей
    invalid_keys = [k for k in keys_flat if k not in INFRASTRUCTURE]
    if invalid_keys:
        raise ValueError(
            f"Неверные ключи: {invalid_keys}. Доступные: {list(INFRASTRUCTURE.keys())}"
        )

    all_services = list(itertools.chain.from_iterable(
        INFRASTRUCTURE[k] for k in keys_flat
    ))

    report = {
        'loaded': [],
        'missing': [],
        'errors': []
    }
    loaded_count = 0

    # Создаем шаблон разреженной матрицы
    combined_matrix = csr_matrix((total_nodes, total_nodes), dtype=np.float64)

    for key in tqdm(all_services, desc=f"Загрузка матриц связей"):
        try:
            base_path = Path(matrix_dir)
            filename = f"{key}_links.parquet"

            file_path = base_path / filename
            updated_path = base_path / "updated" / filename

            # Сначала пробуем загрузить из updated, если флаг включён и файл существует
            if use_updated and updated_path.exists():
                df_relations = pd.read_parquet(updated_path)
                source = "updated"
            elif file_path.exists():
                df_relations = pd.read_parquet(file_path)
                source = "original"
            else:
                report['missing'].append(key)
                continue

            # Фильтруем по границам
            valid_rows = df_relations[
                (df_relations['from'] < total_nodes) &
                (df_relations['to'] < total_nodes)
            ]

            rows = valid_rows['from'].values
            cols = valid_rows['to'].values
            data = valid_rows['demand'].astype(np.float64).values

            current_matrix = csr_matrix((data, (rows, cols)), shape=(total_nodes, total_nodes))
            combined_matrix += current_matrix
            loaded_count += 1
            report['loaded'].append(f"{key} ({source})")

        except Exception as e:
            report['errors'].append(f"Ошибка загрузки {key}: {str(e)}")

    if average and loaded_count > 0:
        combined_matrix /= loaded_count

    return combined_matrix, report

In [6]:
# Расчет метрик для городов
def analyze_mobility(gdf, movement_matrix_csr, anchor_threshold=75):
    """
    Анализ мобильности: самообеспеченность, покрытие опорными пунктами, статистика и потенциальные опорные.

    Parameters:
        gdf (gpd.GeoDataFrame): Информация о городах ['name', 'is_anchor_settlement', 'geometry', 'population']
        movement_matrix_csr (csr_matrix): Разреженная матрица перемещений (from -> to)
        anchor_threshold (float): Порог для потенциальных опорных пунктов (%)

    Returns:
        dict: Результаты анализа
    """
    # Убедимся, что матрица CSR
    movement_matrix_csr = csr_matrix(movement_matrix_csr)

    # Обнулим диагональ (не считаем связи "город-сам_с_собой")
    movement_matrix_csr = movement_matrix_csr.tolil()
    movement_matrix_csr.setdiag(0)
    movement_matrix_csr = movement_matrix_csr.tocsr()

    # Получаем индексы опорных и неопорных городов
    anchor_ids = gdf[gdf['is_anchor_settlement']].index.tolist()
    non_anchor_ids = gdf[~gdf['is_anchor_settlement']].index.tolist()

    results = {}

    # 1. Самообеспеченность и тип города
    self_sufficiency_df, self_sufficiency_pct = compute_self_sufficiency(gdf, movement_matrix_csr)
    results['self_sufficiency'] = self_sufficiency_df

    # 2. Покрытие опорными пунктами
    coverage_df = compute_anchor_coverage(gdf, movement_matrix_csr, anchor_ids, non_anchor_ids)
    results['anchor_coverage'] = coverage_df

    # 3. Статистика по опорным пунктам
    anchor_stats_df = compute_anchor_stats(gdf, coverage_df, self_sufficiency_df, anchor_ids)
    results['anchor_stats'] = anchor_stats_df

    # 4. Потенциальные опорные пункты
    potential_anchors_df = compute_potential_anchors(
        gdf, movement_matrix_csr, self_sufficiency_pct, non_anchor_ids, anchor_threshold
    )
    results['potential_anchors'] = potential_anchors_df

    return results

def compute_self_sufficiency(gdf, movement_matrix_csr):
    """Вычисляет самообеспеченность каждого города."""
    total_outflow = np.array(movement_matrix_csr.sum(axis=1)).flatten()
    population = gdf['population'].values
    self_sufficiency_pct = ((population - total_outflow) / population * 100).round(5)

    city_type = []
    for idx in gdf.index:
        self_pct = self_sufficiency_pct[idx]
        if self_pct > 0:
            inflow = movement_matrix_csr[:, idx].sum()
            if inflow > 0:
                city_type.append("градообразующий")
            else:
                city_type.append("градообслуживающий")
        else:
            city_type.append("не самодостаточный")

    df = pd.DataFrame({
        'city_name': gdf['name'],
        'population': gdf['population'],
        'outflow': total_outflow.round(5),
        'self_sufficiency_pct': self_sufficiency_pct,
        'city_type': city_type
    })

    return df, self_sufficiency_pct

def compute_anchor_coverage(gdf, movement_matrix_csr, anchor_ids, non_anchor_ids):
    """Покрытие неопорных городов опорными."""
    coverage_data = []

    movement_csc = movement_matrix_csr.tocsc()  # Для эффективного доступа по столбцам

    for city_id in non_anchor_ids:
        city_name = gdf.at[city_id, 'name']
        for anchor_id in anchor_ids:
            to_anchor = round(float(movement_csc[city_id, anchor_id]), 5)
            if to_anchor >= 1:
                to_others = round(float(movement_csc[city_id].sum() - to_anchor), 5)
                coverage_pct = round((to_anchor / (to_anchor + to_others)) * 100, 5) if (to_anchor + to_others) > 0 else 0
                coverage_data.append({
                    'city_id': city_id,
                    'city_name': city_name,
                    'anchor_id': anchor_id,
                    'anchor_name': gdf.at[anchor_id, 'name'],
                    'to_anchor': to_anchor,
                    'to_other_non_anchors': to_others,
                    'coverage_pct': coverage_pct
                })

    return pd.DataFrame(coverage_data) if coverage_data else pd.DataFrame()

def compute_anchor_stats(gdf, coverage_df, self_sufficiency_df, anchor_ids):
    """Статистика по опорным пунктам."""
    stats = []
    for anchor_id in anchor_ids:
        anchor_name = gdf.at[anchor_id, 'name']
        anchor_data = coverage_df[coverage_df['anchor_name'] == anchor_name]
        if not anchor_data.empty:
            stats.append({
                'anchor_name': anchor_name,
                'mean_coverage': round(anchor_data['coverage_pct'].mean(), 5),
                'median_coverage': round(anchor_data['coverage_pct'].median(), 5),
                'min_coverage': round(anchor_data['coverage_pct'].min(), 5),
                'max_coverage': round(anchor_data['coverage_pct'].max(), 5),
                'num_covered_cities': len(anchor_data)
            })
        else:
            stats.append({
                'anchor_name': anchor_name,
                'mean_coverage': 0.0,
                'median_coverage': 0.0,
                'min_coverage': 0.0,
                'max_coverage': 0.0,
                'num_covered_cities': 0
            })

    df = pd.DataFrame(stats)

    anchor_self_suff = self_sufficiency_df[
        self_sufficiency_df['city_name'].isin(df['anchor_name'])
    ].set_index('city_name')['self_sufficiency_pct']

    df['is_weak_anchor'] = df['anchor_name'].map(lambda x: anchor_self_suff.get(x, 100) < 95.0)

    return df

def compute_potential_anchors(gdf, movement_matrix_csr, self_sufficiency_pct, non_anchor_ids, threshold):
    """Определение потенциальных опорных пунктов."""
    movement_csc = movement_matrix_csr.tocsc()
    potential = []

    for col_id in non_anchor_ids:
        incoming = movement_csc[:, col_id].toarray().flatten()
        total_incoming = round(incoming.sum(), 5)
        from_non_anchors = incoming[non_anchor_ids].sum()
        self_pct = self_sufficiency_pct[col_id]

        if total_incoming >= 1 and self_pct >= threshold:
            num_sources = int((incoming > 0).sum())

            potential.append({
                'city_id': col_id,
                'city_name': gdf.at[col_id, 'name'],
                'incoming_from_others': total_incoming,
                'from_non_anchors': round(from_non_anchors, 5),
                'from_anchors': round(total_incoming - from_non_anchors, 5),
                'num_sources': num_sources,
                'self_sufficiency_pct': round(self_pct, 5)
            })

    return pd.DataFrame(potential) if potential else pd.DataFrame()

In [7]:
# Генерация карты
def create_anchor_flow_map(
    matrix: csr_matrix,
    gdf: gpd.GeoDataFrame,
    analysis_results: dict,
    polygons_gdf: gpd.GeoDataFrame = None,
    direction: str = 'index',
    default_tiles: str = 'cartodbpositron'
) -> folium.Map:
    
    if direction not in ['index', 'columns']:
        raise ValueError("direction must be either 'index' or 'columns'")
    
    # Центр карты
    anchor_points = gdf[gdf['is_anchor_settlement']]['geometry']
    center = anchor_points.unary_union.centroid

    # Создаем карту
    fmap = folium.Map(
        location=[center.y, center.x], 
        zoom_start=7,
        tiles=default_tiles,
        attr='Map data © OpenStreetMap contributors'
    )

    # Подложки
    folium.TileLayer('openstreetmap', name='OpenStreetMap').add_to(fmap)
    folium.TileLayer('cartodbdark_matter', name='CartoDB Dark Matter').add_to(fmap)

    # Слои
    edges_layer = folium.FeatureGroup(name='Потоки ≥ 1', show=True)
    edges_below_one = folium.FeatureGroup(name='Потоки < 1', show=False)
    cities_with_edges = folium.FeatureGroup(name='Города с потоками', show=True)
    cities_without_edges = folium.FeatureGroup(name='Города без потоков', show=False)
    polygons_layer = folium.FeatureGroup(name='Полигоны', show=True)

    # Полигоны
    if polygons_gdf is not None:
        polygons_gdf = polygons_gdf.to_crs(epsg=4326)
        for _, row in polygons_gdf.iterrows():
            folium.GeoJson(row['geometry'],
                           style_function=lambda x: {
                               'fillColor': 'blue', 'color': 'black', 'weight': 1, 'fillOpacity': 0.3
                           }).add_to(polygons_layer)

    # Справочники
    id_to_geom = gdf['geometry'].to_dict()
    id_to_name = gdf['name'].to_dict()
    is_anchor = gdf['is_anchor_settlement'].to_dict()

    # Легенда цветовой шкалы
    non_zero_values = matrix.data[matrix.data > 0]
    if len(non_zero_values) == 0:
        raise ValueError("Матрица не содержит ненулевых значений")
        
    min_val, max_val = non_zero_values.min(), non_zero_values.max()
    log_min = np.log10(min_val + 1e-10)
    log_max = np.log10(max_val + 1e-10)
    log_range = log_max - log_min
    cmap = plt.get_cmap('RdYlGn_r')
    scalar_map = ScalarMappable(norm=Normalize(vmin=log_min, vmax=log_max), cmap=cmap)

    def get_color(value):
        if value <= 0:
            return "#808080"
        log_val = np.log10(value + 1e-10)
        rgba = scalar_map.to_rgba(log_val)
        return "#{:02x}{:02x}{:02x}".format(
            int(rgba[0]*255), int(rgba[1]*255), int(rgba[2]*255)
        )

    # Tooltip
    self_sufficiency = analysis_results['self_sufficiency'].set_index('city_name')['self_sufficiency_pct']
    anchor_stats = analysis_results.get('anchor_stats', pd.DataFrame())

    def create_tooltip(city_id):
        name = id_to_name[city_id]
        row = analysis_results['self_sufficiency'].set_index('city_name').loc[name]

        tooltip = f"<b>{name}</b><br>"
        tooltip += f"Самообеспеченность: {row['self_sufficiency_pct']:.1f}%<br>"
        tooltip += f"Тип: {row['city_type']}<br>"

        if is_anchor[city_id]:
            tooltip += f"(Опорный пункт)<br>"

            stats = anchor_stats[anchor_stats['anchor_name'] == name]
            if not stats.empty:
                stats = stats.iloc[0]
                if stats['is_weak_anchor']:
                    tooltip += f"<b>Слабый опорный пункт</b><br>"
                tooltip += f"Средняя обеспеченность других: {stats['mean_coverage']:.1f}%<br>"
                tooltip += f"Медианная обеспеченность других: {stats['median_coverage']:.1f}%<br>"
                tooltip += f"Обеспечивает городов: {int(stats['num_covered_cities'])}"
        else:
            pot_df = analysis_results.get('potential_anchors', pd.DataFrame())
            if not pot_df.empty and name in pot_df['city_name'].values:
                tooltip += f"<b>Потенциальный опорный пункт</b><br>"

        return tooltip

    # Обход разреженной матрицы
    cities_with_edges_set = set()
    flow_data = []

    row, col, data = find(matrix)
    for source, target, value in zip(row, col, data):
        if value <= 0:
            continue

        if direction == 'index':
            from_id, to_id = source, target
        else:
            from_id, to_id = target, source

        cities_with_edges_set.add(from_id)
        cities_with_edges_set.add(to_id)
        flow_data.append((from_id, to_id, value))

    # Отрисовка линий
    for from_id, to_id, value in flow_data:
        from_point = id_to_geom[from_id]
        to_point = id_to_geom[to_id]
        line_color = get_color(value)
        line_weight = 1 + min(np.log1p(abs(value)) / 2, 5)
        tooltip_text = f"{id_to_name[from_id]} → {id_to_name[to_id]}: {value:.2f} чел."

        line = folium.PolyLine(
            locations=[(from_point.y, from_point.x), (to_point.y, to_point.x)],
            color=line_color,
            weight=line_weight,
            opacity=0.8,
            tooltip=tooltip_text,
        )

        if abs(value) < 1:
            line.add_to(edges_below_one)
        else:
            line.add_to(edges_layer)

    # Отрисовка городов
    for city_id in gdf.index:
        point = id_to_geom[city_id]
        marker = folium.CircleMarker(
            location=(point.y, point.x),
            radius=4 if is_anchor[city_id] else 2,
            color="red" if is_anchor[city_id] else "green",
            fill=True,
            fill_opacity=0.9,
            tooltip=create_tooltip(city_id),
            popup=create_tooltip(city_id)
        )
        
        if city_id in cities_with_edges_set:
            marker.add_to(cities_with_edges)
        else:
            marker.add_to(cities_without_edges)

    # Добавляем слои
    polygons_layer.add_to(fmap)
    edges_layer.add_to(fmap)
    edges_below_one.add_to(fmap)
    cities_with_edges.add_to(fmap)
    cities_without_edges.add_to(fmap)

    # Легенда
    legend_html = f'''
    <div style="position: fixed; bottom: 50px; left: 50px; width: 200px; height: 180px; 
                border:2px solid grey; z-index:9999; font-size:14px;
                background-color:white; padding: 10px;">
        <b>Легенда (логарифмическая шкала):</b><br>
        <i class="fa fa-circle" style="color:red"></i> Опорные города<br>
        <i class="fa fa-circle" style="color:green"></i> Неопорные города<br>
        <div style="background: linear-gradient(to right, #00ff00, #ff0000); height: 20px;"></div>
        <div style="display: flex; justify-content: space-between;">
            <span>10<sup>{int(log_min)}</sup></span>
            <span>10<sup>{int((log_min+log_max)/2)}</sup></span>
            <span>10<sup>{int(log_max)}</sup></span>
        </div>
        <div style="font-size:12px; color:#555;">Min: {min_val:.0f}, Max: {max_val:.0f}</div>
    </div>
    '''
    fmap.get_root().html.add_child(folium.Element(legend_html))

    # Контроль слоев
    folium.LayerControl(collapsed=False).add_to(fmap)

    return fmap

## 0.3 Загрузка и подготовка данных

### 0.3.1 Данные из и для TownsNet

In [8]:
region = BaseModel()
region.from_pickle('data/lo_region.pickle') # Модель из TownsNet

# region = Region.from_pickle(f'data/lo_region.pickle') # Модель из TownsNet
# INFRASTRUCTURE = region.get_service_types_df().groupby('infrastructure')['name'].unique().to_dict() # Словарь сервисов по группам
# pprint([f'{i:<12}: {list(j)}' for (i, j) in INFRASTRUCTURE.items()], width=150) # Вывод групп
# municipal_districts = region.districts # муниципальные районы ЛО
# towns_df = region.get_towns_gdf() # Города/деревни ЛО
supporting_cities = gpd.read_parquet("data/1_polygons.parquet") # Опорные города ЛО
cities_gdf = gpd.read_parquet('data/cities_gdf.parquet') # Города/деревни ЛО

ueqi_df = region.calculate_ueqi() # Расчет UEQI по новой формуле
# print(f'Need to add capacity: {count_change(ueqi_df, "Мурино", 1)}')

# Создаем словарь для подсчета повторений
name_counts = defaultdict(int)

# Функция для добавления нумерации одноименных городов 
def add_numbering(name):
    name_counts[name] += 1
    return f"{name} {name_counts[name]}" if name_counts[name] > 1 else name

# Применяем к столбцу name
cities_gdf['name'] = cities_gdf['name'].apply(add_numbering)

### 0.3.2 Первичное обновление UEQI для Planter
потому что мы не знаем как подсчитаны старые UEQI для городов

In [9]:
# Загрузка модели
wff = WorkForceFlows.from_pickle('data/wff_1812 new.pkl')

Class instance loaded from data/wff_1812 new.pkl


In [10]:
ueqi_df[ueqi_df.town_name == 'Бокситогорск']

,town_name,population,geometry,capacity_cafe__coffee,capacity_kindergarten,capacity_school,capacity_public_transport_stop,capacity_library,capacity_parking,capacity_railway_station,...,count_car_service,count_fire_station,settlement_name,district_name,ueqi_residential,ueqi_street_networks,ueqi_green_spaces,ueqi_public_and_business_infrastructure,ueqi_social_and_leisure_infrastructure,ueqi_citywide_space
id,,,,,,,,,,,,,,,,,,,,,
121,Бокситогорск,8001,POINT (548021.065 6593118.772),250.0,1500.0,1000.0,1500.0,500.0,2500.0,1250.0,...,0,0,Бокситогорское городское поселение,Бокситогорский муниципальный район,49.994,81.24,46.869,100.0,49.994,9.374


In [11]:
name = "Ленинградская область, Бокситогорск" # Можно узнать все города из wff.cities

# Можно взять параметры из ueqi_df[ueqi_df.town_name == 'Мурино'] ← ВАЖНО! Убедитесь, что вы используете правильное название города без области, но для wff с областью
new_params = { # Первичное обновление UEQI для Planter
    "ueqi_residential":                         49.994,
    "ueqi_street_networks":                     81.24,
    "ueqi_green_spaces":                        46.869,
    "ueqi_public_and_business_infrastructure":  100.0,
    "ueqi_social_and_leisure_infrastructure":   49.994,
    "ueqi_citywide_space":                      9.374,
}

wff.update_city_params(name, new_params)
wff.recalculate_after_update()

Updated parameters for Ленинградская область, Бокситогорск
Recalculating after updating parameters


100%|██████████| 1106/1106 [00:01<00:00, 663.98it/s]


Recalculation complete.


In [12]:
# Выделение городов только по ЛО и сохранение новых значений постоянного населения
area = ox.geocode_to_gdf("Ленинградская область")
lo_cities = wff.cities.clip(area.to_crs(3857)).copy()
lo_cities['population'] = lo_cities['population'] + lo_cities['flows_in'] - lo_cities['flows_out']
lo_cities['region_city'] = lo_cities['region_city'].str.split(', ', expand=True)[1] # У TownsNet названия городов без областей, поэтому нужно удалить их из названия
lo_cities[['region_city','population']].to_csv("data/population.csv", index=False)

### 0.2.3 Обновление населения

In [13]:
# update_population(region.model, towns_df, path='data/population.csv')

population = pd.read_csv('data/population.csv') 
region.update_population(population)

ueqi_df = region.calculate_ueqi()
region[121] # Проверим Бокситогорск

### ====== Принимаем это за базовое население ====== ###

town_name                                               Бокситогорск
population                                                   15849.0
geometry                 POINT (548021.0652682963 6593118.771699116)
capacity_cafe__coffee                                          250.0
capacity_kindergarten                                         1500.0
                                            ...                     
count_railway_station                                              0
count_car_service                                                  0
count_fire_station                                                 0
settlement_name                   Бокситогорское городское поселение
district_name                     Бокситогорский муниципальный район
Name: 121, Length: 95, dtype: object

# 1. Выбор города

In [14]:
name = 'Бокситогорск'
get_ueqi(wff, name) # Просмотр нынешных значений UEQI для города

,region_city,ueqi_residential,ueqi_street_networks,ueqi_green_spaces,ueqi_public_and_business_infrastructure,ueqi_social_and_leisure_infrastructure,ueqi_citywide_space
106,"Ленинградская область, Бокситогорск",49.994,81.24,46.869,100.0,49.994,9.374


# 2. Изменение UEQI

In [15]:
new_params = {
    # "ueqi_residential":                         49.994,
    # "ueqi_street_networks":                     81.24,
    "ueqi_green_spaces":                        56.869, # +10 UEQI
    # "ueqi_public_and_business_infrastructure":  100.0,
    # "ueqi_social_and_leisure_infrastructure":   49.994,
    # "ueqi_citywide_space":                      9.374,
}

# Update the city data in the DataFrame
wff.update_city_params('Ленинградская область, Бокситогорск', new_params)
wff.recalculate_after_update()

# Выделение области
area = ox.geocode_to_gdf("Ленинградская область")
highlighted_cities = wff.cities.clip(area.to_crs(3857)).copy()
highlighted_cities['population'] = highlighted_cities['population'] + highlighted_cities['flows_in'] - highlighted_cities['flows_out']
highlighted_cities['region_city'] = highlighted_cities['region_city'].str.split(', ', expand=True)[1] # У TownsNet названия городов без областей, поэтому нужно удалить их из названия
highlighted_cities[['region_city', 'population']].to_csv("data/population_new.csv", index=False)

Updated parameters for Ленинградская область, Бокситогорск
Recalculating after updating parameters


100%|██████████| 1106/1106 [00:01<00:00, 594.32it/s] 


Recalculation complete.


In [16]:
# Обновим население городов после изменения UEQI
population_new = pd.read_csv('data/population_new.csv') 
region.update_population(population_new)
region[121] # Проверим Бокситогорск

### ====== Принимаем это за новое население ====== ###

town_name                                               Бокситогорск
population                                                   20200.0
geometry                 POINT (548021.0652682963 6593118.771699116)
capacity_cafe__coffee                                          250.0
capacity_kindergarten                                         1500.0
                                            ...                     
count_railway_station                                              0
count_car_service                                                  0
count_fire_station                                                 0
settlement_name                   Бокситогорское городское поселение
district_name                     Бокситогорский муниципальный район
Name: 121, Length: 95, dtype: object

# 3. Расчет матриц переходов по новым UEQI

## 3.1 Расчет количества и добавление необходимого сервиса

In [17]:
# Посмотим, на сколько нужно поднять
change_capacity = count_change_capacity(region.towns, name, 10)
print(f"Изменение capacity на 10% для города {name}: {change_capacity} мест")

Изменение capacity на 10% для города Бокситогорск: 2020 мест


In [18]:
updated_service = region.update_service(name, change_capacity, new_params)
print(f'Случайный обновленный сервис: {updated_service}')

print('======= Таблица для проверки новых сервисов =======')
region.services[(region.services.town.str.contains(name)) & (region.services.service_type == updated_service)]

Случайный обновленный сервис: park
======= Таблица для проверки новых сервисов =======


,town,service_type,geometry,capacity
175,Бокситогорск,park,POINT (547977.638 6593091.073),250
176,Бокситогорск,park,POINT (548120.962 6592967.185),250
177,Бокситогорск,park,POINT (548140.672 6592774.843),250
178,Бокситогорск,park,POINT (548041.259 6593578.502),250
179,Бокситогорск,park,POINT (548116.559 6593224.947),250
180,Бокситогорск,park,POINT (547652.597 6593571.952),250
181,Бокситогорск,park,POINT (547814.714 6593576.322),250
182,Бокситогорск,park,POINT (547745.356 6593398.051),2020


## 3.2 Расчет матриц переходов

In [19]:
provision = Provision(region=region.model)
data_path = f'data/provision/updated'

# Создаем директорию, если её нет
os.makedirs(data_path, exist_ok=True)

# Обновляем service_types в регионе
for service_type in region.model.service_types:
    if service_type.name == updated_service:
        st_name = service_type.name
        # Вычисляем provision для текущего service_type
        d_gdf, s_gdf, t_gdf, l_gdf = provision.calculate(service_type)

        print(f"✔ {st_name:<15} was processed")
        l_gdf.to_parquet(os.path.join(data_path, f'{st_name}_links.parquet'))

✔ park            was processed


# 4. Расчет данных для городов

## 4.1 Чтение матриц переходов

In [20]:
use_updated = False

if use_updated:
    population = pd.read_csv('data/population_new.csv') 
    region.update_population(population)
else:
    population = pd.read_csv('data/population.csv') 
    region.update_population(population)

provision_matrix, report = load_matrices(['RECREATION'], 
                                         total_nodes=region.towns.shape[0], 
                                         matrix_dir='data/provision',
                                         use_updated=use_updated, # Внимание! Если True, использовать новое население. Если False, использовать базовое.
                                         average=True)

rows, cols, values = find(provision_matrix)
print(f"Результат загрузки матриц связей:\n{report}")

NameError: name 'INFRASTRUCTURE' is not defined

In [ ]:
# Теперь создадим DataFrame с from_id, to_id и соответствующими точками
flows = []
for i, j, val in zip(rows, cols, values):
    try:
        from_point = cities_gdf.loc[i, 'geometry'].centroid
        to_point = cities_gdf.loc[j, 'geometry'].centroid
    except KeyError:
        continue
    
    flows.append({
        "from_id": i,
        "to_id": j,
        "from_name": cities_gdf.loc[i, 'name'],
        "to_name": cities_gdf.loc[j, 'name'],
        "demand": val,
        "geometry": LineString([from_point, to_point])
    })

# Создаем GeoDataFrame с потоками между городами
gdf_flows = gpd.GeoDataFrame(flows, geometry='geometry', crs=cities_gdf.crs)

# Создаем GeoDataFrame с дополнительной информацией
index_df = pd.DataFrame(index=range(provision_matrix.shape[0]))
cities_reset = cities_gdf.reset_index(drop=True)[['name', 'is_anchor_settlement', 'geometry']]
matrix_with_cities = index_df.join(cities_reset, how='left')
matrix_with_cities['population'] = region.towns['population'].values # контролируй население, не забывай!
gdf = gpd.GeoDataFrame(matrix_with_cities, geometry='geometry', crs=cities_gdf.crs)
gdf.to_crs(crs='EPSG:4326', inplace=True)

In [ ]:
results = analyze_mobility(gdf=gdf, movement_matrix_csr=provision_matrix, anchor_threshold=75)

# 5. Генерация карты миграции

In [ ]:
fmap = create_anchor_flow_map(provision_matrix, gdf, results)

In [ ]:
fmap